In [ ]:
#|default_exp metadata

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import os
import csv
import json
from glob import glob
from math import ceil
from datetime import datetime
from collections import defaultdict
from typing import Optional, List, Union

from pathlib import Path
import pandas as pd

import numpy as np

from clip_plot.utils import clean_filename, FILE_NAME
from clip_plot.layouts import get_layouts, get_heightmap, get_hotspots
from clip_plot.utils import is_number, get_version, write_json, read_json

# Handle metadata (dates, categories, etc.)

In [ ]:
#| export

def get_metadata_list(meta_dir: str) -> list[dict] | list[str]:
    """Return a list of objects with image metadata.

    Will create 'tags' key if 'category' is in metadata
    but not 'tags'.

    Args:
        metadata (str, default = None): Metadata location

    Returns:
        l (List[dict]): List of metadata

    Notes:
        No check for 'filename' is performed

    Todo:
        Think about separating .csv and json functionality.
        Can we use pandas numpy to process csv?
    """
    # handle csv metadata
    metaList = []
    if meta_dir.endswith(".csv"):
        with open(meta_dir) as f:
            reader = csv.reader(f)
            headers = [i.lower() for i in next(reader)]
            for i in reader:
                metaList.append(
                    {
                        headers[j]: i[j] if len(i) > j and i[j] else ""
                        for j, _ in enumerate(headers)
                    }
                )
    
    # handle json metadata
    else:
        for i in glob(meta_dir, recursive=True):
            with open(i) as f:
                metaList.append(json.load(f))

    # if the user provided a category but not a tag, use the category as the tag
    for metaDict in metaList:
        if "category" in metaDict and ("tags" in metaDict) is False:
            metaDict.update({"tags": metaDict["category"]})
    return metaList, headers

In [ ]:
#| export

def write_metadata(imageEngine):
    """
    Write list `metadata` of objects to disk
    Tightly coupled to ImageEngine class
    """
    metadata = imageEngine.metadata
    data_dir = imageEngine.data_dir
    if not metadata:
        return

    meta_dir = data_dir / "metadata"
    for subdir in ["filters", "options", "file"]:
        (meta_dir / subdir).mkdir(parents=True, exist_ok=True)

    # Create the lists of images with each tag
    d = defaultdict(list)
    for img in imageEngine:
        row = img.metadata
        filename = img.unique_name
        row["tags"] = [j.strip() for j in row.get("tags", "").split("|")]
        for j in row["tags"]:
            d["__".join(j.split())].append(filename)
        write_json((meta_dir/"file"/f"{filename}.json"), row)

    write_json(
        (meta_dir/"filters"/"filters.json"),
        [{"filter_name": "select",
          "filter_values": list(d.keys()),
        }],
    )

    # create the options for the category dropdown
    for i in d:
        write_json((meta_dir/"options"/ f"{i}.json"), d[i])

    # create the map from date to images with that date (if dates present)
    date_d = defaultdict(list)
    for i in metadata:
        date = i.get("year", "")
        if date:
            date_d[date].append(clean_filename(i[FILE_NAME]))

    # find the min and max dates to show on the date slider
    dates = np.array([int(i.strip()) for i in date_d if is_number(i)])
    domain = {"min": float("inf"), "max": -float("inf")}
    mean = np.mean(dates)
    std = np.std(dates)
    for i in dates:
        # update the date domain with all non-outlier dates
        if abs(mean - i) < (std * 4):
            domain["min"] = int(min(i, domain["min"]))
            domain["max"] = int(max(i, domain["max"]))

    # write the dates json
    if len(date_d) > 1:
        write_json(
            (meta_dir/"dates.json"),
            {"domain": domain, "dates": date_d,},
        )

In [ ]:
#| export

def get_manifest(imageEngine, atlas_data,
                 plot_id: str | None,
                 output_dir: Path,
                 has_metadata: bool = False,
                 ):
    """
    Create and return the base object for the manifest output file
    Returns: None
    """
    data_dir = output_dir/"data"
    # store each cell's size and atlas position
    atlas_ids = {i["idx"] for i in atlas_data}
    sizes = [[] for _ in atlas_ids]
    pos = [[] for _ in atlas_ids]
    for i in atlas_data:
        sizes[i["idx"]].append([i["w"], i["h"]])
        pos[i["idx"]].append([i["x"], i["y"]])

    # obtain the paths to each layout's JSON positions
    layouts = get_layouts(imageEngine, **kwargs)
    # create a heightmap for the umap layout
    if "umap" in layouts and layouts["umap"]:
        get_heightmap(layouts["umap"]["variants"][0]["layout"], "umap", **kwargs)

    # specify point size scalars
    point_sizes = {}
    point_sizes["min"] = 0
    point_sizes["grid"] = 1 / ceil(imageEngine.count ** (1 / 2))
    point_sizes["max"] = point_sizes["grid"] * 1.2
    point_sizes["scatter"] = point_sizes["grid"] * 0.2
    point_sizes["initial"] = point_sizes["scatter"]
    point_sizes["categorical"] = point_sizes["grid"] * 0.6
    point_sizes["geographic"] = point_sizes["grid"] * 0.025

    # fetch the date distribution data for point sizing
    if "date" in layouts and layouts["date"]:
        date_layout = read_json(layouts["date"]["labels"])
        point_sizes["date"] = 1 / (
            (date_layout["cols"] + 1) * len(date_layout["labels"])
        )

    # create manifest json
    manifest = {
        "version": get_version(),
        "plot_id": plot_id,
        "output_directory": output_dir.as_posix(),
        "layouts": layouts,
        "initial_layout": "umap",
        "point_sizes": point_sizes,
        "imagelist": (data_dir / "imagelist.json").as_posix(),
        "atlas_dir": (data_dir/"atlases").as_posix(),
        "metadata": has_metadata,
        "default_hotspots": get_hotspots(imageEngine, layouts=layouts,
                                         n_preproc_dims=kwargs["cluster_preproc_dims"],
                                         **kwargs),
        "custom_hotspots": data_dir/"hotspots/user_hotspots.json",
        "gzipped": False,
        "config": {
            "sizes": {
                "atlas": imageEngine.atlas_size,
                "cell": imageEngine.cell_size,
                "lod": imageEngine.lod_cell_height,
            },
        },
        "creation_date": datetime.today().strftime("%d-%B-%Y-%H:%M:%S"),
    }

    # # store parameters that will impact embedding
    # embed_params = ["embed_model", "n_neighbors", "min_dist", "metric", "max_clusters", "min_cluster_size"]
    # for e in embed_params:
    #     manifest.update({e: kwargs[e]})

    write_json(data_dir/"manifests/manifest.json", manifest)
    write_json(data_dir/"/manifest.json", manifest)

    imagelist = {
        "cell_sizes": sizes,
        "images": [img.unique_name for img in imageEngine],
        "atlas": {
            "count": len(atlas_ids),
            "positions": pos,
        },
    }

    write_json(data_dir / "imagelist.json", imagelist)

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()